In [1]:
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools
from spikingjelly.activation_based import neuron, functional, layer, surrogate

C:\Users\SHAURYAM DUBEY\AppData\Local\Temp\ipykernel_39240\1178550895.py:3: DeprecationWarning: The module snntorch.backprop will be deprecated in  a future release. Writing out your own training loop will lead to substantially faster performance.
  from snntorch import backprop


In [3]:
# Define hyperparameters
batch_size = 64

# Load FER2013 dataset
transform = transforms.Compose([
    # transforms.Grayscale(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Change brightness and contrast
    # transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),  # Add small shifts
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load the dataset
train_dataset = datasets.ImageFolder(
    root='./dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(
    root='./dataset/test', transform=transform)
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 12 * 12, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 7)  # FER-2013 has 7 emotion classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [6]:
model = SimpleCNN().to(device)
model.load_state_dict(torch.load('best_model.pth', map_location=device))

C:\Users\SHAURYAM DUBEY\AppData\Local\Temp\ipykernel_39240\3069337107.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth', 

<All keys matched successfully>

In [ ]:
from spikingjelly.activation_based import ann2snn

model_conv = ann2snn.Converter(mode='max', dataloader=train_loader,device=device)

snn_model = model_conv(model).to(device)


100%|██████████| 448/448 [00:14<00:00, 30.72it/s]


In [ ]:
print

graph():
    %x : [num_users=1] = placeholder[target=x]
    %conv_layers_0 : [num_users=1] = call_module[target=conv_layers.0](args = (%x,), kwargs = {})
    %snn_tailor_0_1 : [num_users=1] = call_module[target=snn tailor.0.0](args = (%conv_layers_0,), kwargs = {})
    %snn_tailor_0_2 : [num_users=1] = call_module[target=snn tailor.0.1](args = (%snn_tailor_0_1,), kwargs = {})
    %snn_tailor_0_3 : [num_users=1] = call_module[target=snn tailor.0.2](args = (%snn_tailor_0_2,), kwargs = {})
    %conv_layers_2 : [num_users=1] = call_module[target=conv_layers.2](args = (%snn_tailor_0_3,), kwargs = {})
    %conv_layers_3 : [num_users=1] = call_module[target=conv_layers.3](args = (%conv_layers_2,), kwargs = {})
    %snn_tailor_1_1 : [num_users=1] = call_module[target=snn tailor.1.0](args = (%conv_layers_3,), kwargs = {})
    %snn_tailor_1_2 : [num_users=1] = call_module[target=snn tailor.1.1](args = (%snn_tailor_1_1,), kwargs = {})
    %snn_tailor_1_3 : [num_users=1] = call_module[target=snn t

In [14]:
def evaluate():
    correct = 0
    total = 0
    snn_model.eval()

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            # images = spikegen.rate(images, num_steps=10)
            outputs = snn_model(images)
            # outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')


evaluate()

Test Accuracy: 20.05%
